# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import packages

In [ ]:
from IPython.display import clear_output

# Installation of the necessary packages for FEM (Functional spaces, weak formulations etc):
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"

# mesh generator:

try:
    import gmsh
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"

clear_output(wait = True)
print("Both dolfin and gmsh are installed.")

Both dolfin and gmsh are installed.


In [ ]:
import os
# Insert below the directory in which you have saved this notebook
dir = 'DIR_NAME'
os.chdir(dir)
import geometry
import matplotlib.pyplot as plt
from dolfin import *
import numpy as np
import json

In [ ]:
# Define switch functions
def asfunction(vector,V):
    '''
    Views a vector as a Dolfin function
    :param vector: numpy array
    :param V: dolfin.function.functionspace.FunctionSpace object
    :return: dolfin.function.function.Function object
    '''
    uv = Function(V)
    uv.vector()[:] = vector[dof_to_vertex_map(V)]
    return uv

def asvector(function,mymesh):
    '''
    Views a function as a vector
    :param function: dolfin.function.function.Function object
    :param mymesh: dolfin.cpp.mesh.Mesh object
    :return: numpy array
    '''
    return function.compute_vertex_values(mymesh)

# Advection-Diffusion

We want to solve numerically the equation:

\begin{equation}
\frac{\partial u}{\partial t} - \mu\Delta u + \mathbf{b}\cdot\nabla u= 0
\end{equation}

dove $\mathbf{b}=[1-t,1-t]$. We consider homogeneous Neumann boundary conditions and
\begin{equation}
u_{0}(x,y) = (x-1)^2+(y-1)^2,
\end{equation}

as initial solution.

Applying Backward Euler method we have

\begin{equation}
\frac{u^{n+1} - u^{n}}{\Delta t} - \mu\Delta u^{n+1} + \mathbf{b^{n+1}} \cdot\nabla u^{n+1} = 0,
\end{equation}

which implies

\begin{equation}
\implies u^{n+1} - \Delta t \mu\Delta u^{n+1} + \Delta t\mathbf{b^{n+1}} \cdot \nabla u^{n+1} = u^{n} .
\end{equation}


In [ ]:
# Create folder in which data will be saved
dir_ad = dir + '/AD'
if not os.path.exists(dir_ad):
  os.makedirs(dir_ad)
os.chdir(dir_ad)

In [ ]:
def solveAD(U, T, dt, mu, V, mymesh):
    """
    Solves the 2D advection-diffusion equation using a finite elements scheme
    :param U: Initial condition (numpy array)
    :param T: Final time (float)
    :param dt: Time step (float)
    :param mu: Diffusion coefficient (float)
    :param V: Function Space (dolfin.function.functionspace.FunctionSpace)
    :param mymesh: Mesh (dolfin.cpp.mesh.Mesh)
    :return: U (numpy array)
    """
    Nt = int(T / dt) + 1

    for i in range(Nt - 1):
        t = (i + 1) * dt
        f = asfunction(U[i], V)
        b = Constant((1. - t, 1. - t))  # Advection term

        # Weak Formulation
        u, v = TrialFunction(V), TestFunction(V)  # Solution and Test Function

        # Function that allows fenics to fix boundary condition
        def dir_bound(x, on_boundary):
            return on_boundary

        g = Expression("pow(x[0]-1,2) + pow(x[1]-1,2)", degree=2)
        dbc = DirichletBC(V, g, dir_bound)

        L = u * v * dx + dt * mu * inner(grad(u), grad(v)) * dx + dt * inner(b, grad(u)) * v * dx  # Bilinear Form
        F = f * v * dx  # Forcing term

        # Solver
        u = Function(V)
        solve(L == F, u, bcs=dbc)

        U[i + 1] = asvector(u, mymesh)  # Conversion to discrete vector

    return U

In [ ]:
train_size = 100
num_directions = 4
spd = train_size/num_directions
data = {}
for i in range(train_size):
  # mesh generation
  R = geometry.Rectangle((0,0),(1,1))
  if i<spd:
    C = geometry.Circle((0.5 + i/100,0.5),0.15) 
  elif (i>=spd and i<2*spd):
    C = geometry.Circle((0.5 + (i - spd)/100,0.5 + (i - spd)/100),0.15)
  elif (i>=2*spd and i<3*spd):
    C = geometry.Circle((0.5 - (i - 2*spd)/100,0.5 + (i - 2*spd)/100),0.15)
  else:
    C = geometry.Circle((0.5 - (i - 3*spd)/100,0.5),0.15) 
  name = "geometry" + str(i+1) 
  mymesh = geometry.mesh(R-C, stepsize=0.04,name=name)
  connectivity = mymesh.cells()

  # Construction of P1 Continous Galerkin finite element space
  V = dolfin.function.functionspace.FunctionSpace(mymesh, 'CG', 1)

  x, y = mymesh.coordinates().T # Extract separately mesh coordinates

  T = 2 # Final time
  dt = 0.02  # Time step
  mu = 0.1 # Diffusion coefficient

  Nt = int(T/dt) + 1 # Number of time steps
  Nh = mymesh.num_vertices() # Dofs in space
  U = np.zeros((Nt, Nh)) # Solution matrix
  U[0] =  (x-1)**2 + (y-1)**2 # Initial solution
  U = solveAD(U,T,dt,mu,V,mymesh)

  data[str(i+1)] = {"mesh": name ,"traj":U.tolist()} # save the name of the mesh and the solution
                                                     # in a dictionary

      
with open('data.json', 'w') as f:
  json.dump(data,f)

# Advection-Diffusion + Stokes

In [ ]:
# Create folder in which data will be saved
dir_stokes = dir + '/Stokes'
if not os.path.exists(dir_stokes):
  os.makedirs(dir_stokes)
os.chdir(dir_stokes)

We want to solve a Stokes problem to determine advection field $\mathbf{b}$:
\begin{align}
-\nu \Delta \mathbf{b} + \nabla p &= 0 \qquad \text{in} \ \Omega\\
\nabla \cdot  \mathbf{b} &= 0 \qquad  \text{in} \ \Omega
\end{align}

where p is a pressure field. The boundary conditions are given by:
\begin{align}
\mathbf{b}  &= 0 \qquad \text{on} \ \Gamma_{D} \\
\mathbf{b} &= \mathbf{b_{in}} \quad  \text{on} \ \Gamma_{in}\\
\nu \frac{\partial \mathbf{b}}{\partial \mathbf{n}} - p\mathbf{n} &= 0 \qquad \text{on} \ \Gamma_{N}
\end{align}

where 
\begin{align}
\mathbf{b_{in}} &= (\frac{40Uy(0.5-y)}{0.5^2},0)  \\
U &= 0.3\\
\nu &= 0.001
\end{align}


In [ ]:
def build_space(u_in, mymesh):
    """Prepare data for Stokes problem. Return function
    space and list of boundary conditions.
    :param u_in: Inflow velocity (dolfin.Expression)
    :param mymesh: Mesh (dolfin.cpp.mesh.Mesh)
    :return W: Function space (dolfin.function.functionspace.FunctionSpace)
            bcs: List of boundary conditions (list of dolfin.fem.bcs.DirichletBC)   
    """

    # Define domain
    L = 1.
    W = 0.5

    # Build function spaces (Taylor-Hood)
    P2 = VectorElement("P", mymesh.ufl_cell(), 2)
    P1 = FiniteElement("P", mymesh.ufl_cell(), 1)
    TH = MixedElement([P2, P1])
    W = FunctionSpace(mymesh, TH)

    def inflow(x,on_boundary):
      return on_boundary and (x[0] == 0.)
    def walls(x,on_boundary):
      return on_boundary and ((x[0] != 0) and (x[0] != 1.))

    g = Constant((0.,0.))
    bc_in = DirichletBC(W.sub(0),u_in,inflow)
    bc_walls =  DirichletBC(W.sub(0),g,walls)


    bcs = [bc_walls, bc_in]

    return W, bcs

def solve_stokes(W, nu, bcs):
    """Solve steady Stokes and return the solution
    :param W: Function space (dolfin.function.functionspace.FunctionSpace)
    :param nu: Viscosity (float)
    :param bcs: List of boundary conditions (list of dolfin.fem.bcs.DirichletBC)
    :return u: Velocity (dolfin.function.function.Function)
    """

    # Define variational forms
    u, p = TrialFunctions(W)
    v, q = TestFunctions(W)
    a = nu*inner(grad(u), grad(v))*dx - p*div(v)*dx - q*div(u)*dx
    L = inner(Constant((0, 0)), v)*dx

    # Solve the problem
    w = Function(W)
    solve(a == L, w, bcs)
    u,p = w.split(deepcopy=True)

    return u

Then solve numerically the equation

\begin{equation}
\frac{\partial u}{\partial t} - \mu\Delta u + \mathbf{b}\cdot\nabla u= 0
\end{equation}

using $\mathbf{b}$ found before as advection coefficient and
\begin{equation}
u_{in}(x,y) = (4y(0.5 - y)/(0.5^2))[\mathbb{1}(x=0)],
\end{equation}

as initial solution.

Applying Backward Euler method we have

\begin{equation}
\frac{u^{n+1} - u^{n}}{\Delta t} - \Delta u^{n+1} + \mathbf{b^{n+1}}\nabla u^{n+1} = 0,
\end{equation}

which implies

\begin{equation}
\implies u^{n+1} - \Delta t\Delta u^{n+1} + \Delta t\mathbf{b^{n+1}}\nabla u^{n+1} = u^{n} .
\end{equation}


In [ ]:
def make_tuple(x,y):
    ''' Creates a list of tuples from two vectors
    :param x: vector (numpy array)
    :param y: vector (numpy array)
    :return: list of tuples
    '''
    res = []
    for i in range(x.shape[0]):
        res.append((x[i],y[i]))

    return res

In [ ]:
def solveADbump(U, T, dt, mu, V, u_in, mymesh):
    ''' Solves the 2D advection-diffusion equation with a Stokes advection field
    :param U: Initial condition (numpy array)
    :param T: Final time
    :param dt: Time step
    :param mu: Diffusion coefficient
    :param V:  Function space (dolfin.cpp.function.functionspace.FunctionSpace)
    :param u_in: Inflow velocity (dolfin.function.function.Function)
    :param mymesh: Mesh (dolfin.cpp.mesh.Mesh)
    :return: U (numpy array)
    '''
    Nt = int(T / dt) + 1

    # Solve Stokes
    u_in_stokes = Expression(("40.0*U*x[1]*(0.5 - x[1])/(0.5*0.5)", "0.0"),
                             degree=2, U=0.3)
    nu = Constant(0.001)
    # Prepare function space, BCs
    W, bcs = build_space(u_in_stokes, mymesh)
    b = solve_stokes(W, nu, bcs)

    for i in range(Nt - 1):
        t = (i + 1) * dt
        f = asfunction(U[i], V)

        # Weak Formulation
        u, v = TrialFunction(V), TestFunction(V)

        def inflow(x, on_boundary):
            return on_boundary and (x[0] == 0.)

        def walls(x, on_boundary):
            return on_boundary and (x[0] != 0 and x[0] != 1.)

        g = Constant(0.)
        bc_in = DirichletBC(V, u_in, inflow)
        bc_wi = DirichletBC(V, g, walls)

        bcs = [bc_wi, bc_in]

        L = u * v * dx + dt * mu * inner(grad(u), grad(v)) * dx + dt * inner(b, grad(u)) * v * dx
        F = f * v * dx

        u = Function(V)
        solve(L == F, u, bcs=bcs)

        U[i + 1] = asvector(u, mymesh)

    return U


In [ ]:
train_size=125
step = 0.4/train_size
data = {}
for i in range(train_size):

  #mesh generation
  f = 4.
  width = 0.2
  height = 15
  mean = 0.3 + (step*i)
  x = np.linspace(mean - width, mean + width, 50)
  y = (-np.cos((x-mean)*f*np.pi) + np.cos((width)*f*np.pi))/height +0.5 
  xy = make_tuple(x,y)
  xy.reverse()
  points = [(0.,0.),(1.,0.),(1.,0.5)] + xy + [(0.,0.5),(0.,0.)]
  polygon = geometry.Polygon(points)
  name = "geometry" + str(i+1)
  mymesh = geometry.mesh(polygon,0.035,name=name)


  V = FunctionSpace(mymesh, 'CG', 1)

  x, y = mymesh.coordinates().T 

  T = 0.5 
  dt = 0.01  
  mu = 0.01 # Diffusion coefficient

  Nt = int(T/dt) + 1 
  Nh = mymesh.num_vertices() 
  U = np.zeros((Nt, Nh)) 
  # Problem data
  u_in = Expression("4.0*x[1]*(0.5 - x[1])/(0.5*0.5)",degree=2)
  U[0] = (4.0*y*(0.5 - y)/(0.5**2)) * (x==0) + 0. 


  U = solveADbump(U,T,dt,mu,V,u_in,mymesh)
  U[U<0] = 0.

  data[str(i+1)] = {"mesh": name ,"traj":U.tolist()} # save the name of the mesh and the solution
                                                     # in a dictionary

      
with open('data.json', 'w') as f:
  json.dump(data,f)
